In [1]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Configuration constants
WEAVIATE_URL="https://mmtip1s2qwwe1q3bwih2fw.c0.us-west3.gcp.weaviate.cloud"
WEAVIATE_API_KEY="37Nj5DR1LWMDz98URsr3ib7e80pnHHAoyqn1"
DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [14]:
# Setup vector store
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
# )


client = weaviate.connect_to_local(
                     host="192.168.100.125",
                        port=8080,
                        grpc_port=50051
                )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [15]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True),
)

In [18]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=10,
    alpha=0.5,
)

In [20]:
question = "Đối với mô tô, vi phạm không có giấy chứng nhận đăng ký xe sẽ bị xử phạt như thế nào?"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : mô_tô , gắn máy mức phạt : phạt tiền từ 2.000.000 đồng đến 3.000.000 đồng_nội_dung vi_phạm : điều_khiển xe không có chứng_nhận đăng_ký xe hoặc sử_dụng chứng_nhận đăng_ký xe đã hết hạn sử_dụng ( mô_tô , gắn máy )

Metadata: {'category': 'mô tô, gắn máy', 'fine_amount': 'Phạt tiền từ 2.000.000 đồng đến 3.000.000 đồng', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: mô tô, gắn máy\nMức phạt: Phạt tiền từ 2.000.000 đồng đến 3.000.000 đồng\nNội dung vi phạm: Điều khiển xe không có chứng nhận đăng ký xe hoặc sử dụng chứng nhận đăng ký xe đã hết hạn sử dụng (mô tô, gắn máy)'}

Score: 0.7073780298233032

Result 2:

Text: loại phương_tiện : mô_tô , gắn máy mức phạt : phạt tiền từ 4.000.000 đồng đến 6.000.000 đồng_nội_dung vi_phạm : gắn biển số không đúng với chứng_nhận đăng_ký xe hoặc gắn biển số không do cơ_quan có thẩm_quyền cấp ( mô_tô , gắn máy )

Metadata: {'category': 'mô tô, gắn máy', 'fine_amount': 'Phạt tiền từ 4.000.000 đồng đến 6.000